## Instalando as bibliotecas

In [1]:
!pip install nest_asyncio

## Lendo a tabela geolocation do HIVE

In [31]:
from pyspark.sql import SparkSession
from pyspark.sql import HiveContext

hive_context = HiveContext(sc)

spark = SparkSession \
    .builder \
    .appName("Teste API") \
    .enableHiveSupport() \
    .getOrCreate()

geo = spark.read.orc('/datalake/dadosbrutos/olist_geolocation_dataset.orc')

In [32]:
geo.select('geolocation_zip_code_prefix').distinct().count()

19015

## Carregando os CEPs distintos em uma lista

In [3]:
cep_array = [str(row.geolocation_zip_code_prefix) for row in geo.select('geolocation_zip_code_prefix').distinct().collect()]
print(f"{len(cep_array)} CEPs distintos encontrados no olist_geolocation_dataset.orc")

19015 CEPs distintos encontrados no olist_geolocation_dataset.orc


## Consultando CEPs no site do Correios com apenas os 5 primeiros digitos
* Usando a API pycep conseguimos consultar apenas com o CEP completo, 8 digitos
* Consultando direto no site do Correios conseguimos consultar com 5 digitos e descobrir a cidade e estado do CEP

In [4]:
import asyncio
import time
import aiohttp
import nest_asyncio
import pandas as pd
import json
from pyspark.sql import Row

global URL
# URL do site do correios
URL = 'https://buscacepinter.correios.com.br/app/endereco/carrega-cep-endereco.php'
global ceps_com_erro
ceps_com_erro = []
# Função para pegar o primeiro resultado da pesquisa de CEP com apenas 5 digitos
async def get_address(session, cep):
    async with session.post(url=URL, data={'endereco': cep, 'tipoCEP': 'ALL'}) as response:
        response = await response.text()
        try:
            data = json.loads(response)["dados"][0]
            if data["cep"] != '': 
                data_selected = {
                    "cep": data["cep"],
                    "uf": data["uf"],
                    "cidade": data["localidade"]
                }
                results.append(data_selected)
                print(f"{str(len(results)).zfill(6)} CEPs consultados", end="\r")
        except Exception as e:
            #print(f"ERRO: {e}", end="\r")
            ceps_com_erro.append(cep)
            pass

# Função para criar as tasks assíncronas, uma task para cada cep
async def get_all_addresses(ceps):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for cep in ceps:
            task = asyncio.ensure_future(get_address(session, cep))
            tasks.append(task)
        await asyncio.gather(*tasks, return_exceptions=False)
        

# Função prncipal para iniciar o loop assíncrono e criar o Dataframe com os resultados
def consulta_lote(ceps_array):
    global results
    results = []
    nest_asyncio.apply()
    start_time = time.time()
    asyncio.get_event_loop().run_until_complete(get_all_addresses(ceps_array))
    
    df = spark.createDataFrame((Row(**x) for x in results))
    
    duration = time.time() - start_time
    print(f"Downloaded {len(ceps_array)} ceps in {duration/60} minutes")
    return df

In [5]:
import math
import numpy as np

start_time = time.time()

tamanho_lote = 1000
qtd_lotes = math.floor(len(cep_array)/tamanho_lote)

print(f"Iniciando a consulta de {qtd_lotes} lotes com aprox. {tamanho_lote} ceps cada.")
cep_lotes = np.array_split(cep_array, qtd_lotes)

dataframes = {}
counter = 0
for lote in cep_lotes:
    counter += 1
    print(f"Consultando lote {counter}")
    dataframes[f"df_part{counter}"] = consulta_lote(lote)

duration = time.time() - start_time
print(f"Tempo total da carga: {duration/60} minutos")
print(f"Total de CEPs não encontrados: {len(ceps_com_erro)}")

Iniciando a consulta de 19 lotes com aprox. 1000 ceps cada.
Consultando lote 1
Downloaded 1001 ceps in 0.13712219397226968 minutes
Consultando lote 2
Downloaded 1001 ceps in 0.14149274428685507 minutes
Consultando lote 3
Downloaded 1001 ceps in 0.15122833251953124 minutes
Consultando lote 4
Downloaded 1001 ceps in 0.13631155093510947 minutes
Consultando lote 5
Downloaded 1001 ceps in 0.1362757166226705 minutes
Consultando lote 6
Downloaded 1001 ceps in 0.13811051845550537 minutes
Consultando lote 7
Downloaded 1001 ceps in 0.1695775548617045 minutes
Consultando lote 8
Downloaded 1001 ceps in 0.169659690062205 minutes
Consultando lote 9
Downloaded 1001 ceps in 0.140899391969045 minutes
Consultando lote 10
Downloaded 1001 ceps in 0.1382043917973836 minutes
Consultando lote 11
Downloaded 1001 ceps in 0.15547292629877726 minutes
Consultando lote 12
Downloaded 1001 ceps in 0.1526509722073873 minutes
Consultando lote 13
Downloaded 1001 ceps in 0.12330770095189413 minutes
Consultando lote 14
D

## Unindo os lotes em apenas um dataframe

In [6]:
df_final = dataframes["df_part1"]
#dataframes.pop("df_part1")
for df in dataframes:
    if df == "df_part1": continue
    print(f"Unindo o {df} ao df principal", end = "\r")
    df_final = df_final.union(dataframes[df])

In [7]:
import pyspark.sql.functions as F
df_final = df_final.withColumn('cep_5_digitos', F.col("cep").substr(1,5))
df_final.show(truncate=False)

+--------+---------------------+---+-------------+
|cep     |cidade               |uf |cep_5_digitos|
+--------+---------------------+---+-------------+
|76976970|Primavera de Rondônia|RO |76976        |
|77303970|Rio da Conceição     |TO |77303        |
|45300970|Amargosa             |BA |45300        |
|55445000|Batateira            |PE |55445        |
|45157970|Cândido Sales        |BA |45157        |
|47960970|Angical              |BA |47960        |
|49290970|Itabaianinha         |SE |49290        |
|48370970|Esplanada            |BA |48370        |
|79785971|Angélica             |MS |79785        |
|58819970|Marizópolis          |PB |58819        |
|16250970|Clementina           |SP |16250        |
|89555970|Ipoméia              |SC |89555        |
|45540991|Gongogi              |BA |45540        |
|77930970|Axixá do Tocantins   |TO |77930        |
|83450970|Bocaiúva do Sul      |PR |83450        |
|78888972|Nova Ubiratã         |MT |78888        |
|89669970|Ipira                

## Escrevendo o resultado final no HDFS

In [8]:
df_final.write.orc(f'/datalake/dadosbrutos/geolocation_correios.orc', 'overwrite')